In [10]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    DecimalType,
    TimestampType,
)
from pyspark.sql.functions import create_map, lit
from databricks.connect import DatabricksSession

In [1]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

NameError: name 'dbutils' is not defined

In [11]:
schema = StructType(
    [
        StructField("ride_id", StringType(), True),
        StructField("rideable_type", StringType(), True),
        StructField("started_at", TimestampType(), True),
        StructField("ended_At", TimestampType(), True),
        StructField("start_station_name", StringType(), True),
        StructField("start_station_id", StringType(), True),
        StructField("end_station_name", StringType(), True),
        StructField("end_station_id", StringType(), True),
        StructField("start_lat", DecimalType(), True),
        StructField("start_lng", DecimalType(), True),
        StructField("end_lat", DecimalType(), True),
        StructField("end_lng", DecimalType(), True),
        StructField("member_casual", StringType(), True),
    ]
)

In [12]:
spark = DatabricksSession.builder.getOrCreate()

In [13]:
df = spark.read.csv(f"/Volumes/{catalog}/landing/source_citibike_data", header=True)

In [14]:
df = df.withColumn(
    "metadata",
    create_map(
        lit("pipeline_id"),
        lit(pipeline_id),
        lit("run_id"),
        lit(run_id),
        lit("task_id"),
        lit(task_id),
        lit("processed_timestamp"),
        lit(processed_timestamp),
    ),
)

In [15]:
df.show()

+----------------+-------------+--------------------+--------------------+------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+--------------------+
|         ride_id|rideable_type|          started_at|            ended_at|start_station_name|start_station_id|    end_station_name|end_station_id|         start_lat|         start_lng|           end_lat|           end_lng|member_casual|            metadata|
+----------------+-------------+--------------------+--------------------+------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+--------------------+
|29DAF43DD84B4B7A|electric_bike|2025-03-20 18:58:...|2025-03-20 19:00:...|   6 St & Grand St|           HB302|Mama Johnson Fiel...|         HB404|40.744397833095604|-74.03450086712837| 40.74313993965626|-74.04004096984863|    

In [17]:
df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    f"{catalog}.bronze.jc_citibike"
)